### Code

In [ ]:
import sys

sys.path.append("../")

from typing import Literal

import numpy as np
import pandas as pd
from IPython.display import Markdown
from ipywidgets import interact
from tqdm.auto import tqdm


from dynalign.experiments.paths import KNOWLEDGE_TRANSFER_STUDY_PATH
from dynalign.experiments.defaults import DATASETS

In [ ]:
def get_results_table(
    df: pd.DataFrame,
    dataset: str,
    evaluation_snapshot: Literal["prev", "current", "next"],
) -> pd.DataFrame:
    parsed_data = {}
    df = df[df.dataset == dataset].drop("dataset", axis=1)
    for snapshot_id in df.snapshot_id.unique():
        snapshot_df = df[df.snapshot_id == snapshot_id].drop("snapshot_id", axis=1)
        snapshot_df = snapshot_df[
            snapshot_df.evaluation_snapshot == evaluation_snapshot
        ].drop("evaluation_snapshot", axis=1)
        snapshot_df = snapshot_df.groupby("method").agg(["mean"]).drop("run", axis=1)
        snapshot_df = (snapshot_df).round(
            2
        )  # / snapshot_df.loc["Node2Vec"] * 100).round(2)
        parsed_data[snapshot_id] = snapshot_df["auc"]["mean"].to_dict()

    parsed_df = pd.DataFrame(parsed_data)
    parsed_df.columns.name = "snapshot"
    return parsed_df

In [ ]:
df = pd.DataFrame(pd.read_pickle(KNOWLEDGE_TRANSFER_STUDY_PATH))

### Tables with results

In [ ]:
@interact
def display_result_tables(ds=df.dataset.unique()):
    for evaluation_snapshot in ("prev", "current", "next"):
        display(Markdown(evaluation_snapshot))
        display(
            get_results_table(
                df=df, dataset=ds, evaluation_snapshot=evaluation_snapshot
            ).style.highlight_max()
        )

### Plots

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import qualitative

In [ ]:
methods_to_plot = ["Node2Vec", "Node2VecAligned_L2_ALL", "PosthocALL"]

In [ ]:
fig = make_subplots(rows=2, cols=len(DATASETS), subplot_titles=DATASETS)

for plot_row_id, evaluation_snapshot in enumerate(("prev", "next")):

    for plot_column_id, ds in enumerate(DATASETS):

        data = get_results_table(
            df, dataset=ds, evaluation_snapshot=evaluation_snapshot
        ).T[methods_to_plot]

        for method_id, (method_name, method_df) in enumerate(data.T.iterrows()):
            fig.add_trace(
                go.Scatter(
                    x=method_df.index,
                    y=method_df.values,
                    mode="markers",
                    name=method_name,
                    marker={
                        "symbol": ["circle", "diamond", "square"][method_id],
                        "size": 20,
                        "color": qualitative.Set2[
                            method_id
                        ],  # split('(')[1].split(')')[0]},0.5)",
                    },
                    opacity=1,
                    showlegend=not bool(plot_column_id) and plot_row_id == 0,
                ),
                row=plot_row_id + 1,
                col=plot_column_id + 1,
            )

fig.show()